In [1]:
from IPython.display import display, Markdown

In [2]:
# do bm25 first then vector? nope. maybe just bm25

In [3]:
from sub_agents import knowledge_agent
# knowledge agent can answer questions from  multiple databases
# when the model gets smarter - i only need retrieval, no rag chain needed within the knowledge function

/Users/arjun/Documents/github/smart-agent/gemini-agent/data/ec_sop.csv
csv present
/Users/arjun/Documents/github/smart-agent/gemini-agent/data/sop_docs.csv
csv present


In [7]:
query = "noticed there is discrpency when validating trade, what to do?"

In [8]:
answer = knowledge_agent.execute_task(query)

calling sop_assistant...
noticed there is discrpency when validating trade what to do

Document Numbe
If you have noticed a discrepancy when validating a trade, you should follow the standard operating procedure outlined in the "Validating Trade Details" document. Here are the steps you should take:

1. **Escalate the Discrepancies**:
   - Document any discrepancies or issues you have identified during the validation process in the Trade Validation Log.
   - Next, escalate the discrepancies to the appropriate team or subject matter expert for resolution.
   - Follow up on the resolution and update the Trade Validation Log accordingly.

2. **Record Validation**:
   - Once the trade details have been validated and any discrepancies resolved, record the validation in the Trade Validation Log.
   - Include essential information such as Trade ID, Validation date, Validator's name, Outcome (Validated or Discrepancy), Discrepancy details (if applicable), and Resolution details (if applicable)

In [9]:
Markdown(answer)

If you have noticed a discrepancy when validating a trade, follow these steps:

1. Document the discrepancy in the Trade Validation Log.
2. Escalate the discrepancy to the appropriate team or subject matter expert for resolution.
3. Follow up on the resolution and update the Trade Validation Log accordingly.
4. Once the discrepancy has been resolved, record the validation in the Trade Validation Log.

Remember to perform trade validation on a daily basis, or as per the established trade processing schedule.

In [7]:
from openai import OpenAI
client = OpenAI()

In [8]:
def chat_openai(prompt):
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an assistant who help answer user's question based on given inormation."},
        {"role": "user", "content": f"{prompt}"}
    ]
    )

    return completion.choices[0].message

In [9]:
answer = chat_openai("hello")

In [11]:
answer.content

'Hello! How can I assist you today?'